# How to use pythreejs to plot a superellipsoid

A superellipsoid is given by a parametric function and the equation is very similar to an ellipse equation. We only have different exponents which give us different shapes. For more informations: https://en.wikipedia.org/wiki/Superellipsoid.

The idea of this example is to construct the mesh of the square $[0, 1]\times[0,1]$ and to do a projection of these points on the superillipse which is the 2D shape and then to do a spherical product to have the 3D shape.

In [1]:
import numpy as np

In [2]:
def superellipsoid(n, rx, ry, rz, m1, m2):
    """
    superellipsoid formula with the spherical product of two superellipse
    and update of the global coords array
    
    Parameters
    ----------
    rx : the radius in the x direction 
    ry : the radius in the y direction 
    rz : the radius in the z direction 
    m1 : the exponent of the first superellipse
    m2 : the exponent of the second superellipse
    """
    
    x_box = np.concatenate((np.linspace(-1, 1., n), np.ones(n-2), np.linspace(1, -1., n), -np.ones(n-2)))
    y_box = np.concatenate((-np.ones(n-1), np.linspace(-1, 1., n), np.ones(n-2), np.linspace(1, -1., n-1, endpoint=False)))
    nx_box = x_box.size

    coords = np.empty((nx_box**2, 3), dtype=np.float32)
    
    def superellipse(rx, ry, m):
        """
        superellipse formula with the projection of the unit square

        Parameters
        ----------
        rx : the radius in the x direction 
        ry : the radius in the y direction 
        m : the exponent of the superellipse

        Output
        ------
        the coordinates of the superellipse
        """
        return x_box*rx*(1. - .5*np.abs(y_box)**(2./m))**(m/2.),  y_box*ry*(1. - .5*np.abs(x_box)**(2./m))**(m/2.)

    gx, gy = superellipse(1, 1, m2)
    hx, hy = superellipse(1, 1, m1)
    
    coords[:, 0] = rx*(gx[np.newaxis, :]*hx[:, np.newaxis]).flatten()
    coords[:, 1] = ry*(gx[np.newaxis, :]*hy[:, np.newaxis]).flatten()
    coords[:, 2] = rz*(gy[np.newaxis, :]*np.ones(hx.size)[:, np.newaxis]).flatten()
    
    return coords

In [3]:
# superellipsoid parameters
n = 20
rx = ry = rz = 1.
m1 = m2 = 1.

coords = superellipsoid(n, rx, ry, rz, m1, m2)

In [4]:
from pythreejs import *
from IPython.display import display

view_width = 600
view_height = 400

vertices = BufferAttribute(
    array=coords,
    normalized=False)

meshGeometry = BufferGeometry(
    attributes={
        'position': vertices,
    }
)

In [5]:
texture = ImageTexture('disc.png')

In [6]:
pointsMaterial = PointsMaterial(
    color='#80bfff',
    map=texture,
    size=.1,
    alphaTest=0.5,
)

In [7]:
points = Points(
    geometry=meshGeometry,
    material=pointsMaterial    
)

In [8]:
camera = PerspectiveCamera(
    position=[2, 2, 1],
    aspect=view_width / view_height,
    near=1,
    far=1000,
    children=[PointLight("0xffffff", 0 )])

In [9]:
controls = OrbitControls(
    controlling=camera,
)

In [10]:
ambient_light = AmbientLight('0x222222')

In [11]:
group = Group()

In [12]:
scene = Scene(
    children=[camera, ambient_light, AxesHelper(1), group],
    background = 'black',
)

In [13]:
group.add(points)

In [14]:
renderer = Renderer(
    camera=camera,
    scene=scene,
    controls=[controls],
    width=view_width, 
    height=view_height)

In [15]:
spin_track = NumberKeyframeTrack(name='.rotation[y]', times=[0, 1000], values=[0, 100])
spin_clip = AnimationClip(tracks=[spin_track])
spin_action = AnimationAction(AnimationMixer(group), spin_clip, group)

/home/loic/miniconda3/envs/pythreejs/lib/python3.6/site-packages/ipydatawidgets/ndarray/serializers.py:62: UserWarning: Cannot serialize (u)int64 data, Javascript does not support it. Casting to (u)int32.
  warnings.warn('Cannot serialize (u)int64 data, Javascript does not support it. '


In [16]:
from ipywidgets import IntSlider, FloatSlider, HBox, VBox

n_slider, m1_slider, m2_slider = (
    IntSlider(description='n', min=5, max=50, step=1, value=n,
              continuous_update=False, orientation='vertical'),
    FloatSlider(description='m1', min=0.01, max=4.0, step=0.01, value=m1,
                continuous_update=False, orientation='vertical'),
    FloatSlider(description='m2', min=0.01, max=4.0, step=0.01, value=m2,
                continuous_update=False, orientation='vertical')
)

In [17]:
rx_slider, ry_slider, rz_slider = (FloatSlider(description='rx', min=0.01, max=10.0, step=0.1, value=rx, 
                                               continuous_update=False, orientation='horizontal'),
                                   FloatSlider(description='ry', min=0.01, max=10.0, step=0.1, value=ry, 
                                               continuous_update=False, orientation='horizontal'),
                                   FloatSlider(description='rz', min=0.01, max=10.0, step=0.1, value=rz, 
                                               continuous_update=False, orientation='horizontal'))

In [18]:
def update(change):
    coords = superellipsoid(n_slider.value, rx_slider.value, ry_slider.value, rz_slider.value, 
                   m1_slider.value, m2_slider.value)
    vertices.array = coords
    
n_slider.observe(update, names=['value'])
m1_slider.observe(update, names=['value'])
m2_slider.observe(update, names=['value'])
rx_slider.observe(update, names=['value'])
ry_slider.observe(update, names=['value'])
rz_slider.observe(update, names=['value'])

In [21]:
VBox([HBox([renderer, 
            VBox([HBox([m1_slider, m2_slider, n_slider]), spin_action])
           ]),
      rx_slider, 
      ry_slider,
      rz_slider])

VBox(children=(HBox(children=(Renderer(camera=PerspectiveCamera(aspect=1.5, children=(PointLight(color='0xffffff', intensity=0.0, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), far=1000.0, near=1.0, position=(3.373094094168995, 2.7715244561523895, -3.885084509024818), projectionMatrix=(1.4296712803397058, 0.0, 0.0, 0.0, 0.0, 2.1445069205095586, 0.0, 0.0, 0.0, 0.0, -1.002002002002002, -1.0, 0.0, 0.0, -2.002002002002002, 0.0), quaternion=(-0.08557285113612151, 0.9083355747804472, 0.2290879575961793, 0.3392970356798315), scale=(1.0, 1.0, 1.0), type='PerspectiveCamera', up=(0.0, 1.0, 0.0)), controls=[OrbitControls(controlling=PerspectiveCamera(aspect=1.5, children=(PointLight(color='0xffffff', intensity=0.0, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), far=1000.0, near=1.0, position=(3.373094094168995, 2.7715244561523895, -3.885084509024818), projectionMatrix=(1.4296712803397058, 0.0, 0.0, 0.0, 0.0, 2.1445069205095586, 0.0, 0.0, 0.0, 0.0, -1.002002002002002, -1.0, 0.0, 0.0, -2.002002002002002, 0.0), quaternion=(-0.08557285113612151, 0.9083355747804472, 0.2290879575961793, 0.3392970356798315), scale=(1.0, 1.0, 1.0), type='PerspectiveCamera', up=(0.0, 1.0, 0.0)))], scene=Scene(background='black', children=(PerspectiveCamera(aspect=1.5, children=(PointLight(color='0xffffff', intensity=0.0, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), far=1000.0, near=1.0, position=(3.373094094168995, 2.7715244561523895, -3.885084509024818), projectionMatrix=(1.4296712803397058, 0.0, 0.0, 0.0, 0.0, 2.1445069205095586, 0.0, 0.0, 0.0, 0.0, -1.002002002002002, -1.0, 0.0, 0.0, -2.002002002002002, 0.0), quaternion=(-0.08557285113612151, 0.9083355747804472, 0.2290879575961793, 0.3392970356798315), scale=(1.0, 1.0, 1.0), type='PerspectiveCamera', up=(0.0, 1.0, 0.0)), AmbientLight(color='0x222222', quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='AmbientLight', up=(0.0, 1.0, 0.0)), AxesHelper(quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='LineSegments', up=(0.0, 1.0, 0.0)), Group(children=(Points(geometry=BufferGeometry(attributes={'position': BufferAttribute(array=array([[ 1.0228853 ,  0.9930925 , -0.99654025],
       [ 0.9205967 ,  0.8937833 , -1.        ],
       [ 0.81830823,  0.794474  , -1.        ],
       ...,
       [ 1.03      ,  0.8968862 , -0.6975782 ],
       [ 1.03      ,  0.8968862 , -0.7972322 ],
       [ 1.03      ,  0.8968862 , -0.8968862 ]], dtype=float32), normalized=False, version=17)}, type='BufferGeometry'), material=PointsMaterial(alphaTest=0.5, color='#80bfff', map=ImageTexture(imageUri='disc.png', repeat=(1.0, 1.0), version=1), size=0.1, type='PointsMaterial'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Points', up=(0.0, 1.0, 0.0)),), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Group', up=(0.0, 1.0, 0.0))), fog=None, overrideMaterial=None, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Scene', up=(0.0, 1.0, 0.0)), shadowMap=WebGLShadowMap()), VBox(children=(HBox(children=(FloatSlider(value=0.01, continuous_update=False, description='m1', max=4.0, min=0.01, orientation='vertical', step=0.01), FloatSlider(value=0.01, continuous_update=False, description='m2', max=4.0, min=0.01, orientation='vertical', step=0.01), IntSlider(value=21, continuous_update=False, description='n', max=50, min=5, orientation='vertical'))), AnimationAction(clip=AnimationClip(duration=1000.0, tracks=(NumberKeyframeTrack(name='.rotation[y]', times=array([   0, 1000]), values=array([  0, 100])),)), localRoot=Group(children=(Points(geometry=BufferGeometry(attributes={'position': BufferAttribute(array=array([[ 1.0228853 ,  0.9930925 , -0.99654025],
       [ 0.9205967 ,  0.8937833 , -1.        ],
       [ 0.81830823,  0.794474  , -1.        ],
       ...,
       [ 1.03      ,  0.8968862 , -0.6975782 ],
       [ 1.03      ,  0.8968862 , -0.7972322 ],
       [ 1.03      ,  0.8968862 , -0.8968